# Builds and saves a model for later use based on claim and not claim dataset


Author: Kiran Bhattacharyya

Revision: 5/11/18 - DRM - translate .py files into .ipynb, misc formatting

In [1]:
# import relevant libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
import pickle

### load data


In [2]:
allFeatureName = pd.read_pickle('../Data/allFeatureNames.pkl') # feature names
needCiteFeat = np.load('../Data/NeedCiteFeatMat.npy') # feature matrix of claim dataset
notClaimFeat = np.load('../Data/NotClaimFeatMat.npy') # feature matrix of not claim dataset


### concatenate all data

In [3]:
allFeats = np.concatenate((needCiteFeat, notClaimFeat), 0)
allClass = np.concatenate((np.ones((len(needCiteFeat), 1)), np.zeros((len(notClaimFeat), 1))), 0)
needCiteFeat = list() # clear data to save memory
notClaimFeat = list()

I need to fix this bug in the saved data. NOTE THAT IT SHOULD BE value 2823

In [4]:
# feature number 2353 is ']' and is irrelevant, an error in data pre-processing
allFeats[:,2353] = 0


### reorder elements


In [5]:
reOrder = np.random.permutation(len(allFeats))
allFeats = allFeats[reOrder,:]
allClass = allClass[reOrder,:]
y = np.ravel(allClass)


### perform random forest classification to get feature importances


In [6]:
clf = ExtraTreesClassifier(n_estimators=100,n_jobs=-2) #change n_jobs depending on cores
clf.fit(allFeats, y)
featImp = clf.feature_importances_ # get feature importances
featImpSort = np.argsort(featImp) # sort feature importances


In [15]:
featImpSort[0:10]

array([2353, 3046, 1142,  842, 2667, 2677,  647, 3355, 1377, 2473])

In [21]:
allFeats[0:3]

array([[0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 1., 0.]])

### rearrange dimensions so they are sorted by feature importance


In [7]:
allFeats_sort = np.zeros((len(allFeats), int(allFeats.shape[1])))
featName_sort = list()
for i in range(0,len(featImpSort)):
    indx = featImpSort[-(i + 1)]
    allFeats_sort[:,i] = allFeats[:,indx]
    featName_sort.append(allFeatureName.FeatName[indx])


### build a model with only 600 predictors since we found that accuracy does not change that much after the first 600 most important predictors


In [8]:
subFeats = allFeats_sort[:, 0:599]
subFeatName = featName_sort[0:599]
myModel = ExtraTreesClassifier(n_estimators=10,)
myModel.fit(subFeats, y)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

## Save Model

In [9]:
## save the model to disk
filename = '../Data/finalized_model.sav'
pickle.dump(myModel, open(filename, 'wb'))

## save the feature names, feature matrix, and class labels used to create the model
np.save('../Data/Features_finalized_model.npy', subFeats)
np.save('../Data/ClassLabels_finalized_model.npy', y)
subFeatArr = np.array(subFeatName)
np.save('../Data/FeatureName_finalized_model.npy', subFeatArr)
